In [2]:
import pandas as pd
import numpy as np

In [36]:
reclamos = pd.read_excel('./COELCHA_Oficio_Circular_6621.xlsx')

reclamos.drop(reclamos.columns[0], axis=1, inplace=True)

reclamos.columns = np.array(reclamos.iloc[0])

reclamos.drop(reclamos[reclamos[reclamos.columns[0]] == reclamos.columns[0]].index, axis=0, inplace=True)

null_columns = reclamos.columns[reclamos.isnull().sum() > 0]

fechas = reclamos['mm/yyyy'].unique()

reclamos = reclamos.reset_index(drop=True)

reclamos[null_columns] = reclamos[null_columns].fillna(method='ffill', limit=2)

#reclamos.isnull().sum()

reclamos['Fecha'] = pd.to_datetime(reclamos['mm/yyyy']).dt.date

reclamos = reclamos.drop(columns=['Nombre Empresa', 'mm/yyyy'])

columns = reclamos.columns

reclamos[columns[1:6]] = reclamos[columns[1:6]].astype(float)

float_columns = reclamos.select_dtypes(float).columns

reclamos[float_columns] = reclamos[float_columns].round(3)

reclamos.to_csv('reclamos_clean.csv', index=False)

reclamos.info()

reclamos


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90 entries, 0 to 89
Data columns (total 12 columns):
 #   Column                                                                   Non-Null Count  Dtype  
---  ------                                                                   --------------  -----  
 0   Tipo Requerimiento                                                       90 non-null     object 
 1   Eficiencia servicio  Comercial (Eficiencia)                              90 non-null     float64
 2   Eficacia servicio  Comercial (Eficiencia)                                90 non-null     float64
 3   Oportunidad del servicio Comercial (OP)                                  90 non-null     float64
 4   Tiempo medio de resolucion de reclamos, consultas y solicitudes (TRR)    90 non-null     float64
 5   Indicador de nivel de reclamo, consulta y solicitud (INR)                90 non-null     float64
 6   Indicador de facturacion emitida (IFE)                                   90 

,Tipo Requerimiento,Eficiencia servicio Comercial (Eficiencia),Eficacia servicio Comercial (Eficiencia),Oportunidad del servicio Comercial (OP),"Tiempo medio de resolucion de reclamos, consultas y solicitudes (TRR)","Indicador de nivel de reclamo, consulta y solicitud (INR)",Indicador de facturacion emitida (IFE),Indicador de facturacion emitida con consumo estimado (IFEEs),Errores en emision de facturas por errores de lectura (EFErt),Errores en emision de facturas por errores distintos al de lectura (EF),Pagos mal imputados (PMI),Fecha
0,Reclamo,95.990,96.150,93.680,24.000,0.075,100,77.970,0.986,1.000,0.994,2021-02-01
1,Consulta,97.430,100.000,100.000,1.000,0.117,100,77.970,0.986,1.000,0.994,2021-02-01
2,Solicitud,97.060,99.870,95.050,24.000,0.177,100,77.970,0.986,1.000,0.994,2021-02-01
3,Reclamo,96.370,96.380,92.900,23.000,0.083,100,75.050,0.988,1.000,0.994,2021-03-01
4,Consulta,98.110,100.000,100.000,1.000,0.149,100,75.050,0.988,1.000,0.994,2021-03-01
...,...,...,...,...,...,...,...,...,...,...,...,...
85,Consulta,100.000,100.000,100.000,1.000,0.164,100,84.690,0.986,0.998,0.996,2023-06-01
86,Solicitud,39.770,99.960,96.800,5.024,0.146,100,84.690,0.986,0.998,0.996,2023-06-01
87,Reclamo,27.564,94.684,92.691,7.000,0.015,100,85.208,0.987,0.999,0.995,2023-07-01
88,Consulta,100.000,100.000,100.000,2.000,0.168,100,85.208,0.987,0.999,0.995,2023-07-01


In [70]:
fallas = pd.read_csv('./fallas.csv')

fallas['Periodo'] = pd.to_datetime(fallas['Periodo']).dt.date

fallas = fallas.drop(columns=['Nombre Empresa'])

# Convert 'Response Time' column to a consistent format (HH:MM:SS)
def convert_response_time(time_str):
    if '/' in time_str:
        days, time = time_str.split('/')
        hours, minutes = time.split(':')
        seconds = '0'
    else:
        parts = time_str.split(':')
        if len(parts) == 2:
            hours, minutes = parts[0], parts[1]
            seconds= '0'
            days = '0'
        else:
            hours, minutes, seconds = parts[0], parts[1], parts[2]
            days = '0'
    
    return f"{hours.zfill(2)}:{minutes.zfill(2)}:{seconds.zfill(2)}"

fallas['Tiempo promedio concurrencia ante eventos'] = fallas['Tiempo promedio concurrencia ante eventos'].apply(convert_response_time)

def parse_percentage(percentage_str):
    if ',' in percentage_str:
        return float(percentage_str.replace(',' , '.').rstrip('%'))
    return float(percentage_str.rstrip('%'))

fallas['Porcentaje concurrencia mayor a 4 [Hr]'] = fallas['Porcentaje concurrencia mayor a 4 [Hr]'].apply(parse_percentage)
fallas['Porcentaje concurrencia menor a 4 [Hr]'] = fallas['Porcentaje concurrencia menor a 4 [Hr]'].apply(parse_percentage)
fallas['Porcentaje falsas alarmas'] = fallas['Porcentaje falsas alarmas'].apply(parse_percentage)

fallas.to_csv('fallas_clean.csv', index=False)

fallas.info()

fallas

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12 entries, 0 to 11
Data columns (total 11 columns):
 #   Column                                         Non-Null Count  Dtype  
---  ------                                         --------------  -----  
 0   Periodo                                        12 non-null     object 
 1   Cantidad Eventos                               12 non-null     int64  
 2   Número avisos de clientes por eventos (total)  12 non-null     int64  
 3   Tiempo promedio concurrencia ante eventos      12 non-null     object 
 4   Hora de Concurrencia Menor a 4 [Hr]            12 non-null     int64  
 5   Hora de Concurrencia Mayor a 4 [Hr]            12 non-null     int64  
 6   Porcentaje concurrencia menor a 4 [Hr]         12 non-null     float64
 7   Porcentaje concurrencia mayor a 4 [Hr]         12 non-null     float64
 8   Cantidad Eventos Operación Remota              12 non-null     int64  
 9   Cantidad Eventos Operación Manual              12 non-nu

,Periodo,Cantidad Eventos,Número avisos de clientes por eventos (total),Tiempo promedio concurrencia ante eventos,Hora de Concurrencia Menor a 4 [Hr],Hora de Concurrencia Mayor a 4 [Hr],Porcentaje concurrencia menor a 4 [Hr],Porcentaje concurrencia mayor a 4 [Hr],Cantidad Eventos Operación Remota,Cantidad Eventos Operación Manual,Porcentaje falsas alarmas
0,2023-07-01,43,261,00:51:00,43,0,100.0,0.0,0,43,0.0
1,2023-06-01,61,362,01:31:00,55,6,90.2,9.8,0,61,0.0
2,2023-05-01,66,338,00:51:00,65,1,98.5,1.5,0,66,0.0
3,2023-04-01,63,493,03:13:00,53,10,84.1,15.9,0,63,0.0
4,2023-03-01,44,251,01:18:00,40,4,91.0,9.0,0,44,0.0
5,2023-02-01,93,618,03:16:57,70,23,75.0,25.0,0,93,0.0
6,2023-01-01,64,437,01:28:06,60,4,94.0,6.0,0,64,0.0
7,2022-12-01,76,330,01:29:51,73,3,96.0,4.0,0,76,0.0
8,2022-11-01,60,260,01:34:20,57,3,95.0,5.0,0,60,0.0
9,2022-10-01,76,362,01:50:53,68,8,89.0,11.0,0,76,0.0


In [73]:
llamados = pd.read_csv('./llamados.csv')

llamados = llamados.drop(columns='Nombre Empresa')

llamados['Porcentaje de llamadas perdidas (LLP%)'] = llamados['Porcentaje de llamadas perdidas (LLP%)'].apply(parse_percentage)

def parse_month_year(month_year_int):
    date_str = str(month_year_int)
    year = date_str[-4:]
    month = date_str[:-4]
    return pd.to_datetime(f'{year}-{month}-01', format='%Y-%m-%d')

llamados['Periodo'] = llamados['Periodo'].apply(parse_month_year).dt.date

llamados.to_csv('llamados_clean.csv', index=False)

llamados.info()

llamados

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 10 columns):
 #   Column                                                                                       Non-Null Count  Dtype  
---  ------                                                                                       --------------  -----  
 0   Periodo                                                                                      43 non-null     object 
 1   Número total de llamadas recibidas por mes (NLLR)                                            43 non-null     int64  
 2   Número total de llamadas atendidas en IVR por mes (NRIVR)                                    43 non-null     int64  
 3   Número total de llamadas desistidas en IVR por mes (NDIVR)                                   43 non-null     int64  
 4   Número total de llamadas atendidas por ejecutivo por mes (NREJC)                             43 non-null     int64  
 5   Número total de llamadas cursadas por 

,Periodo,Número total de llamadas recibidas por mes (NLLR),Número total de llamadas atendidas en IVR por mes (NRIVR),Número total de llamadas desistidas en IVR por mes (NDIVR),Número total de llamadas atendidas por ejecutivo por mes (NREJC),Número total de llamadas cursadas por gestión de abandono por mes (NGA),Porcentaje de llamadas perdidas (LLP%),"Tasa de ocupación de las líneas, registradas con una frecuencia de una hora (OC)","Tiempo promedio de espera, considerando las llamadas atendidas por un ejecutivo (TPE)","Tiempo de conversación promedio, considerando las llamadas atendidas por un ejecutivo (TPC)"
0,2023-07-01,2907,700,0,2207,478,16.44,0,00:05:09,00:07:24
1,2023-06-01,3785,925,0,1972,554,14.63,0,00:04:51,00:07:09
2,2023-05-01,3342,816,0,1958,902,26.98,0,00:04:28,00:06:58
3,2023-04-01,4280,938,0,2285,1057,24.69,0,00:04:56,00:06:48
4,2023-03-01,3556,806,0,2163,587,16.50,0,00:04:33,00:06:14
5,2023-02-01,4078,981,0,2709,438,10.74,0,00:04:06,00:07:24
6,2023-01-01,3914,791,0,2416,707,18.06,0,00:04:27,00:06:42
7,2022-12-01,3522,788,0,1938,796,22.60,0,00:05:16,00:06:28
8,2022-11-01,2933,680,0,1680,549,19.53,0,00:06:48,00:06:49
9,2022-10-01,3240,746,0,1994,338,15.43,0,00:06:18,00:05:25
